In [ ]:
from dataclasses import dataclass
@dataclass
class FakeTokenizer:
    vocab = ["management"]

from text2network.datasets.text_datasets import *

config = TrainDataConfig(
    data_path="data/preprocessed/",
    new_word_cutoff=5,
    val_items=5
)
pts = PreTrainDataSet(config)
dataset = pts.make_dataset("2000", val_items=0)
tokenizer = FakeTokenizer()


In [ ]:
pts.compute_length("2022")

In [ ]:
i

In [ ]:
dataset

In [ ]:
dd = pts._get_missing_freq_table(llm="2022", tokenizer=tokenizer)
dd

In [ ]:
dd = pts.get_missing_tokens(llm_list=[pts.llms[0]], tokenizer=tokenizer)
dd

In [ ]:
dd = pts.get_missing_tokens(llm_list=pts.llms, tokenizer=tokenizer)
dd

In [4]:
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    llm="2022",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40)

In [ ]:
from text2network.functions.train_mlm import train_mlm_simple


train_mlm_simple(config)

In [5]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)


from transformers import AutoModelForMaskedLM, AutoTokenizer


trainer.prep_mlm_and_tokenizer()
test_model = AutoModelForMaskedLM.from_pretrained(config.llm.model_name_or_path)
model = AutoModelForMaskedLM.from_pretrained(trainer.model_base_folder)
assert model.bert.embeddings.word_embeddings.num_embeddings > test_model.bert.embeddings.word_embeddings.num_embeddings

test_tokenizer = AutoTokenizer.from_pretrained(trainer.tokenizer_base_folder)
negtest_tokenizer = AutoTokenizer.from_pretrained(config.llm.model_name_or_path)
assert test_tokenizer.vocab != negtest_tokenizer.vocab
assert test_tokenizer.vocab["manager"]==negtest_tokenizer.vocab["manager"]

INFO:t2n:Pre-Loading Data and Populating tokenizers
INFO:t2n:Tokenizer vocabulary 30522 items.
DEBUG:t2n:De-Wordpieced Tokenizer has 30522 tokens.
INFO:t2n:Found 2 directories in data path.
INFO:t2n:Found 2 LLMs to train.
3it [00:02,  1.32it/s]
2it [00:01,  1.26it/s]
INFO:t2n:Found 2 missing tokens to be added to the tokenizer
INFO:t2n:After adding missing terms: Tokenizer vocabulary 30524 items.
INFO:t2n:Saving tokenizer to /Users/ingomarquart/Documents/GitHub/text2network/data/output/base_model/tokenizer.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of

In [ ]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, max_steps=10, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)

trainer.train_one_model(llm="2022")

from transformers import AutoModelForMaskedLM, AutoTokenizer

output_folder = trainer.compute_outputfolder_for_llm(config, llm="2022")
test_model = AutoModelForMaskedLM.from_pretrained(output_folder)
test_tokenizer = AutoTokenizer.from_pretrained(output_folder)
assert test_model.bert.embeddings.word_embeddings.num_embeddings == len(test_tokenizer.vocab)

In [8]:
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

data_config = TrainDataConfig(
    data_path="data/preprocessed/",
    shuffle_buffer_size=100,
    new_word_cutoff=5,
    val_items=5,
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config,llm=llm_config, gpu_batch=100,eval_steps=50, max_steps=500, max_seq_length=40, output_folder="data/output/")

trainer = model_trainer(config)


trainer.train_one_model(llm="2022")

INFO:t2n:Starting training for model 
Base model: /Users/ingomarquart/Documents/GitHub/text2network/data/output/base_model/model 
Base tokenizer: /Users/ingomarquart/Documents/GitHub/text2network/data/output/base_model/tokenizer 
Trained model output to: /Users/ingomarquart/Documents/GitHub/text2network/data/output/llms/2022 

/opt/homebrew/Caskroom/miniforge/base/envs/text2network/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  2%|▏         | 11/500 [00:17<12:37,  1.55s/it]

KeyboardInterrupt: 

In [6]:
from dataclasses import asdict
from text2network.training.bert_trainer import model_trainer
from config.dataclasses import LLMConfig, TrainingConfig, TrainDataConfig

import yaml

def save_to_yaml(dataclass_instance, filename):
    with open(filename, 'w') as f:
        yaml.dump(asdict(dataclass_instance), f)
data_config = TrainDataConfig(
    data_path="data/preprocessed/",
)
llm_config = LLMConfig(
    model_name_or_path="bert-base-uncased",
    resume_from_checkpoint=None
)

config = TrainingConfig(data=data_config, llm=llm_config)

In [ ]:
save_to_yaml(config, "config/training.yaml")